In [1]:
import cv2
import pandas as pd
import numpy as np

def preprocessing(val=True):
    
    data = pd.read_csv('data/x_train_gr_smpl.csv')
    labels = pd.read_csv('data/y_train_smpl.csv')

    if(val):
        ## ---------------- Data preparation ---------------- ##
        X_train = []
        for i in range(data.shape[0]):
            img = np.uint8(data.iloc[i])
            edited = cv2.Canny(img, 10, 30)
            edited = cv2.GaussianBlur(edited, (5, 5), 0)
            X_train.append(edited.reshape((1,-1))[0])

        data = pd.DataFrame(X_train)
        ## -------------------------------------------------- ##

    return data, labels

In [2]:
from sklearn.cluster import KMeans
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split
from sklearn.metrics import homogeneity_score

In [3]:
n_clusters = [10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 50, 100]

data, labels = preprocessing()
labels = np.reshape(labels.values, newshape=(1, -1))

for n in n_clusters:
    estimator = KMeans(n_clusters=n, max_iter=50000, random_state=1, n_jobs=-1).fit(data)
    print(f'Number of clusters: {n} Homogeneity score: {homogeneity_score(labels[0], estimator.labels_)}')

Number of clusters: 10 Homogeneity score: 0.4950284618523185
Number of clusters: 11 Homogeneity score: 0.48341617311681173
Number of clusters: 12 Homogeneity score: 0.5167613058592444
Number of clusters: 13 Homogeneity score: 0.5204136214297174
Number of clusters: 14 Homogeneity score: 0.5530250196404715
Number of clusters: 15 Homogeneity score: 0.5568376753630506
Number of clusters: 16 Homogeneity score: 0.5833472105057467
Number of clusters: 17 Homogeneity score: 0.5763995636211059
Number of clusters: 18 Homogeneity score: 0.5815075350489741
Number of clusters: 19 Homogeneity score: 0.6004386672475748
Number of clusters: 20 Homogeneity score: 0.6146503165589815
Number of clusters: 50 Homogeneity score: 0.6949833069037257
Number of clusters: 100 Homogeneity score: 0.7471942791979834
